In [1]:
import tkinter as tk
from tkinter import filedialog
import unicodedata
import pyperclip
import numpy as np

#imports from the custom chord predictor libarary: 
from jl_dictionaries import Dictionaries
from jl_encoding import Encoder




In [2]:

vocab_size = len(Dictionaries.get_all_possible_chords())
encoder = Encoder(Dictionaries.get_all_possible_chords())

Category to number dictionary length: 804
Number to category dictionary length: 804


### 15-2-23 issues: 
- Currently, only chords from the buttoned array can be transposed.
- Chords must be separated by at least a single separator and not sit on the far end of the chords line.
- Need to check what happens when verse is copied and there are two consecutive chord lines with no lyric between them. The code assumes the chords are in the odd lines.
- The code doesn't allow to paste lyrics that preexist with code.
- Should we somehow add automatic parsing for lines longer than 35? Tab4u editor forces lines to be shorter, and forces the user to manually edit them to comply.
- did not empricialy tested the network to make sure its good (it seems decent but no tests have been done, also should look into the code of the original to see how he used a decoder (as his output is 805 long while his chord list is only 804))

In [3]:
from keras.models import load_model
model = load_model('lstm_normalised__W_20_lr_0_0005_epochs=50_batch_128.h5')
#get model encoder related info: 
vocab_size = len(Dictionaries.get_all_possible_chords())
encoder = Encoder(Dictionaries.get_all_possible_chords())
#model = load_model('lstm_normalised__W_20_lr_0_1_epochs=50_batch_128.h5')


Category to number dictionary length: 804
Number to category dictionary length: 804


In [4]:
#define a 9 by 12 matrix of all common chords and thier 9 common modifications: 
basic=['A','Bb','B','C','C#','D','Eb','E','F','F#','G','Ab']
minors=[b+'m' for b in basic]
basic_7s=[b+'7' for b in basic]
basic_m7s=[b+'m7' for b in basic]
basic_maj7s=[b+'maj7' for b in basic]
basic_m7b5=[b+'m7b5' for b in basic]
basic_dim7=[b+'dim7' for b in basic]
basic_4=[b+'4' for b in basic]
basic_add9=[b+'add9' for b in basic]
allchords=[basic,minors,basic_7s,basic_m7s,basic_maj7s,basic_m7b5,basic_dim7,basic_4,basic_add9]
print(allchords)

[['A', 'Bb', 'B', 'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab'], ['Am', 'Bbm', 'Bm', 'Cm', 'C#m', 'Dm', 'Ebm', 'Em', 'Fm', 'F#m', 'Gm', 'Abm'], ['A7', 'Bb7', 'B7', 'C7', 'C#7', 'D7', 'Eb7', 'E7', 'F7', 'F#7', 'G7', 'Ab7'], ['Am7', 'Bbm7', 'Bm7', 'Cm7', 'C#m7', 'Dm7', 'Ebm7', 'Em7', 'Fm7', 'F#m7', 'Gm7', 'Abm7'], ['Amaj7', 'Bbmaj7', 'Bmaj7', 'Cmaj7', 'C#maj7', 'Dmaj7', 'Ebmaj7', 'Emaj7', 'Fmaj7', 'F#maj7', 'Gmaj7', 'Abmaj7'], ['Am7b5', 'Bbm7b5', 'Bm7b5', 'Cm7b5', 'C#m7b5', 'Dm7b5', 'Ebm7b5', 'Em7b5', 'Fm7b5', 'F#m7b5', 'Gm7b5', 'Abm7b5'], ['Adim7', 'Bbdim7', 'Bdim7', 'Cdim7', 'C#dim7', 'Ddim7', 'Ebdim7', 'Edim7', 'Fdim7', 'F#dim7', 'Gdim7', 'Abdim7'], ['A4', 'Bb4', 'B4', 'C4', 'C#4', 'D4', 'Eb4', 'E4', 'F4', 'F#4', 'G4', 'Ab4'], ['Aadd9', 'Bbadd9', 'Badd9', 'Cadd9', 'C#add9', 'Dadd9', 'Ebadd9', 'Eadd9', 'Fadd9', 'F#add9', 'Gadd9', 'Abadd9']]


In [5]:
def init_chords_options(available_chords_list_option):
    global chords_options_list
    operations_list=['delete','copy_row','paste_row','copy_verse','paste_verse']

    common_simple_chords=['A','B','C','D','E','F','G']
    common_simple_chords=[[c+'',c+'m',c+'#',c+'b',c+'#m',c+'7',c+'bm'] for c in common_simple_chords]
    common_simple_chords =[chord for chord_types in common_simple_chords for chord in chord_types if chord in encoder.number_to_category_dict.values()]    
    if available_chords_list_option=='restricted':   
        #define what chords we wish to work with: (there are 804 options, but we only work with basic ones)
        chords_options_list = common_simple_chords
        #temp - remove chords that are not in the chords list: 
        print(chords_options_list)
    elif available_chords_list_option=='common':
        chords_options_list=list(encoder.number_to_category_dict.values())
        #get rid of rare chords (such as those containing 13s, 6s, and so on: )
        chords_ids_to_remove=['13','+','6','sus','add','#9','#11','3','7-9','9-5','11','AmMaj','7-5','aug']
        chords_options_list=[chord for chord in list(encoder.number_to_category_dict.values()) if not any(substring in chord for substring in chords_ids_to_remove)]
        #put the more common chords first in the list:
        # first, remove them from the entire list:  
        for chord in common_simple_chords: 
            chords_options_list.remove(chord)
        #then concatinate them (the common ones are first now) 
        chords_options_list=operations_list+common_simple_chords+chords_options_list 
    elif available_chords_list_option=='all':
        chords_options_list=list(encoder.number_to_category_dict.values())
        #put the more common chords first in the list:
        # first, remove them from the entire list:  
        for chord in common_simple_chords: 
            chords_options_list.remove(chord)
        #then concatinate them (the common ones are first now) 
        chords_options_list=operations_list+common_simple_chords+chords_options_list    
    else: 
        Exception('type method is non valid (must be either "all" or "restricted"')
    return chords_options_list

In [6]:
def pad_sequence(input, length):
    return [0] * (length - len(input)) + input

def get_predictions(model,prev_chords,allowed_chords,top_n=10):
    sequence=[encoder.to_number(chord) for chord in prev_chords if (chord in encoder.category_to_number_dict.keys())]
    if len(sequence)>20:  #if we allready have more than 20 chords, just take the most recent 20 chords
        padded_sequence=sequence[-20:]
    else: 
        padded_sequence=pad_sequence(sequence,20)

    predicted_p_unsorted=model(np.array([padded_sequence]))
    predicted_p_unsorted_array=np.array(predicted_p_unsorted)
    descending_sorting_inds=np.argsort(-1*predicted_p_unsorted).flatten() #negative so sorting will result in descending order
    sorted_predicted_ps=(np.sort(-1*predicted_p_unsorted)*-1).flatten()

    allowed_chords_in_order=[ind for ind in descending_sorting_inds if (not(ind==0) and (encoder.number_to_category_dict[ind] in allowed_chords))]
    allowed_chords_proba_in_order=[predicted_p_unsorted_array[0][ind] for ind in descending_sorting_inds if (not(ind==0) and (encoder.number_to_category_dict[ind] in allowed_chords))]

    top_n_chords=[encoder.number_to_category_dict[chord] for chord in allowed_chords_in_order[0:top_n]]
    top_n_chords_p=allowed_chords_proba_in_order[0:top_n]
    return top_n_chords,top_n_chords_p

top_n_chords,top_n_chords_p=get_predictions(model=model,prev_chords=['C','F','F#','A','G','A','G']*6,allowed_chords=['G','F#','C'],top_n=2)
print('predicted chords:',top_n_chords)
print('predicted probabilities:',top_n_chords_p)

predicted chords: ['C', 'G']
predicted probabilities: [0.16737138, 0.040277347]


In [11]:
copied_line=[]
added_chords=[]
empty_chord_space_symbol='_'
max_words_in_sentence=8
how_many_cords_before_resorting_list=1
hebrew_flag=False
start=[]
end=[]
copied_verse_in_lines=[]
copied_verse_start=[]
copied_verse_end=[]

chords_options_list=init_chords_options('common')

def is_hebrew(term):
    return 'HEBREW' in unicodedata.name(term.strip()[0])


def get_ordered_added_chords_from_text(curr_text_string,empty_chord_space_symbol):
    lines=curr_text_string.split('\n')
    added_cords_from_text=[]
    chords_per_line=[]
    for line in lines: 
        if line.count(empty_chord_space_symbol)>3: #need to make sure this condition makes sense 
            line_chords=[c for c in line.split(empty_chord_space_symbol) if len(c)>0]
            if hebrew_flag: 
                line_chords=line_chords[::-1]
            added_cords_from_text=added_cords_from_text+line_chords
            chords_per_line.append(line)
    return  added_cords_from_text,chords_per_line


def on_double_click(event,widget):
    global chords_options_list,added_chords,copied_line,start,end,copied_verse_in_lines,copied_verse_start,copied_verse_end,used_chord_lines
    if widget=='chords_list':
        curr_chord = chords_list.get(chords_list.curselection())
    elif widget=='suggested_chords_list':
        curr_chord = suggested_chords_list.get(suggested_chords_list.curselection())
        curr_chord=curr_chord.split(' ')[0]
    elif widget=='chord_table':
        print('selection from chord_table')
        curr_chord=event.widget.cget("text")
    elif widget=='used_chord_lines':
        curr_lines=used_chord_lines.get(used_chord_lines.curselection())
        selected_used_chords_line=curr_lines
        curr_chord='paste_from_used_rows'

    curr_text_string=output_text.get('1.0',tk.END)
    added_chords,_=get_ordered_added_chords_from_text(curr_text_string,empty_chord_space_symbol) #this line might be useless as we call it again at the end of the function
    chord_length = len(curr_chord)

    print('selected chord:',curr_chord)
    print('current state of chords list added by user',added_chords)
    print('last chord length in list:',chord_length)
    #print(f'selected chord {chord} length {chord_length}')

    #get the chord left to the cursor by looking at the text string: 
    line_num, col_num = output_text.index('insert').split('.')
    print('selection location on the grid',(line_num, col_num))
    text_string_lines=curr_text_string.split('\n')
    line_num=int(line_num)-1
    col_num=int((int(col_num)-1))
    print('location_on_string:',(line_num,col_num))
    text_string_lines[line_num][col_num]
    print('selected text line string',text_string_lines[line_num])
    for m in np.arange(col_num,-1,-1):
        if text_string_lines[line_num][m]==empty_chord_space_symbol:
            break
    chord_left_to_selection_name_from_string=text_string_lines[line_num][m+1:col_num+1]
    print('chord left of cursor: ',chord_left_to_selection_name_from_string)

    #delete the previous chord from the same row it was inserted at
    if curr_chord=='delete': #delete the chord on the left: 
        chord_length=len(chord_left_to_selection_name_from_string)
        output_text.delete("insert -{}c".format(chord_length), "insert")
        print('state of chord list before deleting: ',added_chords) 
        print('last chord length=',chord_length)
        output_text.insert(tk.INSERT, empty_chord_space_symbol*chord_length)
    elif curr_chord=='copy_row':
        output_text.tag_remove("highlight", 1.0, "end")
        copied_line=text_string_lines[line_num]
        line_len=len(copied_line)
        print('copied the current chord row:',copied_line)
        output_text.tag_add("highlight",str(line_num+1)+'.0',str(line_num+1)+'.'+str(line_len))
    elif curr_chord=='paste_row':
        if len(copied_line)>0:
            curr_line=text_string_lines[line_num]
            line_len=len(curr_line)
            output_text.delete(str(line_num+1)+'.0',str(line_num+1)+'.'+str(line_len))
            output_text.insert(str(line_num+1)+'.0', copied_line)
    elif curr_chord=='copy_verse':
        #get the end verse location ()
        copied_verse_in_lines=extract_chords_from_verse(curr_text_string,start,end)
        copied_verse_start=int(start.split('.')[0])
        copied_verse_end=int(end.split('.')[0])
        output_text.tag_remove("highlight", 1.0, "end")
        copied_line=text_string_lines[line_num]
        line_len=len(copied_line)
        print('copied the current verse:',copied_verse_in_lines)
        output_text.tag_add("highlight",start.split('.')[0]+'.0',end.split('.')[0]+'.'+str(line_len))
    elif curr_chord=='paste_verse':
        if len(copied_verse_in_lines)>0:
            current_cursor_line=line_num
            copied_text_num_lines=len(copied_verse_in_lines)
            temp_text_lines=text_string_lines[line_num:line_num+copied_text_num_lines]
            line_len=len(temp_text_lines[0])
            #take the chord lines from the copied_verse_lines: 
            temp_text_lines[0::2]=copied_verse_in_lines[0::2]
            temp_text_lines_string='\n'.join(temp_text_lines)
            output_text.delete(str(line_num+1)+'.0',str(line_num+copied_text_num_lines)+'.'+str(line_len))
            output_text.insert(str(line_num+1)+'.0', temp_text_lines_string)       

            #get the chords in that line: 
            temp_line_split=copied_line.split(empty_chord_space_symbol)
            chords_in_line=[c for c in temp_line_split if len(c)>0]
            #add the chords: note that order is not currently maintained 
    elif curr_chord=='paste_from_used_rows':
            if not ('scrollable list' in selected_used_chords_line):
                curr_line=selected_used_chords_line
                line_len=len(curr_line)
                output_text.delete(str(line_num+1)+'.0',str(line_num+1)+'.'+str(line_len))
                output_text.insert(str(line_num+1)+'.0', curr_line)
    else:
        #replacement of the chord to the left: 
        if len(chord_left_to_selection_name_from_string)>0: #altough the user didnt ask for delete operation, the cursor is placed right next to another chord so we will replace it
            
            chord_length=len(chord_left_to_selection_name_from_string)
            print(f'replacing {chord_left_to_selection_name_from_string} with {curr_chord}')
            curr_chord=empty_chord_space_symbol*(len(chord_left_to_selection_name_from_string)-len(curr_chord))+curr_chord
            print('after adding markers:',curr_chord)
            chord_length=max(chord_length,len(curr_chord))
        #regular addition of chord:
        output_text.delete("insert -{}c".format(chord_length), "insert")
        output_text.insert(tk.INSERT, curr_chord)

    #get the updated state of the main text outout: 
    curr_text_string=output_text.get('1.0',tk.END)

    added_chords,chords_per_line=get_ordered_added_chords_from_text(curr_text_string,empty_chord_space_symbol)
    
    print('reordering suggested chords list')
    top_n_chords,top_n_chords_p=get_predictions(model=model,prev_chords=added_chords,allowed_chords=chords_options_list,top_n=15)
    print('predicted chords',top_n_chords)
    suggested_chords_list.delete(0,len(chords_options_list))
    chords_options_list1=top_n_chords
    for curr_chord,chord_p in zip(chords_options_list1,top_n_chords_p):
        insertion_chord_and_probability=f'{curr_chord} - {chord_p:.3f}%'
        suggested_chords_list.insert(tk.END, insertion_chord_and_probability)

    #check all currently unique chord lines and add them to the used_chord_lines list if they are not allready there: 
    cur_vals_in_chord_lines=list(used_chord_lines.get(0,tk.END))
    new_used_chords_line=[cur_vals_in_chord_lines[0]] #keep the header: 
    #add all currently pasted lines to the new line list: 
    for curr_chords_per_line in chords_per_line: 
        if len(curr_chords_per_line.strip(empty_chord_space_symbol))>0:
            new_used_chords_line.append(curr_chords_per_line)

    #make sure to keep the header (which is not a chords line): 
    new_used_unique_line=[cur_vals_in_chord_lines[0]]+list(np.unique(new_used_chords_line[1:]))
    used_chord_lines.delete(0,tk.END)        
    for cur_line in new_used_unique_line:
        used_chord_lines.insert("end",cur_line)


def on_paste_and_parse():
    global hebrew_flag
    text = pyperclip.paste()
    parsed_text = ""
    lines = text.split("\n")
    if len(lines)==1:
        words = text.split(" ")
        parsed_w_text = ""
        for w_cnt,word in enumerate(words):
            parsed_w_text = parsed_w_text +' '+ word
            if np.mod(w_cnt+1,max_words_in_sentence)==0:
                parsed_w_text=parsed_w_text+'\n' 
        lines = parsed_w_text.split("\n")

    largest_string=max(lines,key=len)
    #print('largest string:',largest_string)
    largest_string=len(largest_string)
    words_on_longest_line=lines[lines.index(max(lines,key=len))].split(' ')
    max_num_words_in_sentence=len(words_on_longest_line)
    #print('max words in longest setence:',type(max_num_words_in_sentence))
    
    #check if the language is hebrew: 
    hebrew_flag=any([is_hebrew(word) for word in words_on_longest_line if len(word)>1])
    if hebrew_flag: 
        print('detected hebrew')
        hebrew_flag
    else: 
        print('hebrew not detected')
    
    empty_places_for_chords=largest_string+max_num_words_in_sentence
    parsed_text=empty_places_for_chords*empty_chord_space_symbol + '\n'
    for line in lines:
        if len(line)==0: #empty char, add a whole empty string symbols:
            line=empty_places_for_chords*empty_chord_space_symbol + '\n'
        if hebrew_flag:
            parsed_text += empty_places_for_chords*empty_chord_space_symbol + '\n' + line + "\n"
        else:
            parsed_text += line + "\n" + empty_places_for_chords*empty_chord_space_symbol + '\n'

    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, parsed_text)


def save_text():
    edited_text = output_text.get("1.0", "end") # Get the text from the text entry widget
    if not empty_chord_space_symbol==' ':
        edited_text=edited_text.replace(empty_chord_space_symbol,' ')
    print(edited_text)
    
    # Use a dialog box to get the file name to save to
    file_name = filedialog.asksaveasfilename(defaultextension=".txt",initialdir = ".\chord sheets outputs",filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")])

    if file_name:
        with open(file_name, "w") as f:
            f.write(edited_text) # Write the text to a file
        print("Text saved successfully.")
    else:
        print("Save operation cancelled.")




#these 3 functions are built to give the start and end location of the current max text selection
def start_selection(event):
    global start
    start=output_text.index("@%d,%d" % (event.x, event.y))

def continue_selection(event):
    global start,end
    end = output_text.index("@%d,%d" % (event.x, event.y))

def end_selection(event):
    global start,end
    if len(end)==0:
        end=start
    temp=[start,end]
    print('selection start-end: ',(start,end))

def extract_chords_from_verse(curr_text_string,start,end):
    print('entered_verse_parser:')
    start_line_ind=int(start.split('.')[0])-1
    end_line_ind=int(end.split('.')[0])
    curr_text_string_lines=curr_text_string.split('\n')
    copied_verse_in_lines=curr_text_string_lines[start_line_ind:end_line_ind]
    return copied_verse_in_lines

def transponse_chord(curr_chord,direction):
    row,col=np.where(curr_chord==np.array(allchords))
    transposed=curr_chord
    if len(col)>0: 
        col=col[0]
        row=row[0]

        if direction=='up':
            col=col+1
            if col>=len(allchords[0]):
                col=0
        elif direction=='down':
            col=col-1
            if col<0:
                col=len(allchords[0])-1
        transposed=allchords[row][col]   
    else: 
        print(f'chord is not in the buttoned array thus currnetly cannot be transposed:{curr_chord}')

    return transposed

def on_transpose_click(event,widget,direction):
    print('got into the transpose function')
    if start==end:
        change_all_text=True
        curr_text_string=output_text.get('1.0',tk.END)
        s='1'
        e=tk.END
    else: 
        change_all_text=False
        s=str(min(int(start.split('.')[0]),int(end.split('.')[0])))
        e=str(max(int(start.split('.')[0]),int(end.split('.')[0])))
        e_col_largest=str(max(int(start.split('.')[1]),int(end.split('.')[1])))
        curr_text_string=output_text.get(s+'.0',e+'.'+e_col_largest)
    
    if 'transpose' in widget:

        transposed_text=[]
        for line in curr_text_string.split('\n'):  
            #tranposing line by line to avoid accidently taking word parts and replacing them (for this i also add the "empty_chord_space_symbol in the lookup")
            #this means that every chord must be seperated by atleast one space for it to work correctly: 

            if line.count(empty_chord_space_symbol)>2: #its proably a chord line as regular lines have spaces and not empty chord seperators
                line_chords=[c for c in line.split(empty_chord_space_symbol) if len(c)>0] 
                transposed_line_chords=[transponse_chord(chord,direction) for chord in line_chords]
                substitutions_dict={c+'_':t+'_' for c,t in zip(line_chords,transposed_line_chords)}
                for old, new in substitutions_dict.items():
                    line = line.replace(old, new)

            transposed_text.append(line)
        
        transposed_text='\n'.join(transposed_text)
        if change_all_text: 
            output_text.delete('1.0',tk.END)
            output_text.insert('1.0',transposed_text) 
        else: 
            output_text.delete(s+'.0',e+'.'+e_col_largest)
            output_text.insert(s+'.0',transposed_text)     
        
#main editing window: 
root = tk.Tk()
root.title("Guitar Chords")
width  = int(root.winfo_screenwidth()/2)
height = int(root.winfo_screenheight()*0.85)
root.geometry(f'{width}x{height}+10+{int(height*0.025)}')

#######one chord list########
chords_frame = tk.Frame(root)
chords_frame.pack(side="left", fill="both", expand=True)

scrollbar = tk.Scrollbar(chords_frame, orient="vertical")
scrollbar.pack(side="left", fill="y")

chords_label=tk.Label(chords_frame,text='chords\nlist',height=2, width=8 , fg='black')
chords_label.configure(font=("TkFixedFont", 12, "bold"))
#sugested_chords=tk.Label(chords_frame,text='suggested\nchords',height=2, width=9)
#sugested_chords.configure(font=("TkFixedFont", 12, "bold"))
chords_list = tk.Listbox(chords_frame, xscrollcommand=scrollbar.set, height=1, width=10,bg='lavender',cursor="plus",highlightcolor='red',selectmode='SINGLE')

for chord in chords_options_list:
    chords_list.insert(tk.END, chord)
chords_label.pack(side='top',anchor='nw')
chords_list.pack(side="left", fill="both", expand=True)
scrollbar.config(command=chords_list.yview)

######second chord list#########
chords_frame1 = tk.Frame(root)
chords_frame1.pack(side="left", fill="both", expand=True)
scrollbar1 = tk.Scrollbar(chords_frame1, orient="vertical")
chords_label1 = tk.Label(chords_frame1,text='suggested\nchords',height=2, width=8 , fg='black')
chords_label1.configure(font=("TkFixedFont", 12, "bold"))
suggested_chords_list = tk.Listbox(chords_frame1, xscrollcommand=scrollbar1.set, height=1, width=15,bg='lavender blush',cursor="arrow")
for chord in []:
    suggested_chords_list.insert(tk.END, chord)

chords_label1.pack(side='top',anchor='nw')
suggested_chords_list.pack(side="left", fill="both", expand=True)
scrollbar1.config(command=suggested_chords_list.yview)
chords_list_num_selected=np.zeros([len(chords_options_list)])
chords_list.bind("<Double-Button-1>", lambda event: on_double_click(event, "chords_list"))
suggested_chords_list.bind("<Double-Button-1>",  lambda event: on_double_click(event, "suggested_chords_list"))

#main text area: 
output_frame = tk.Frame(root)
output_frame.pack(side="bottom", fill="both", expand=True)
output_text = tk.Text(output_frame, font=("TkFixedFont", 14),cursor="arrow",undo=True,autoseparators=True,maxundo=-1)
#define what happens when marking portion of the text: 
output_text.bind("<Button-1>", start_selection)
output_text.bind("<B1-Motion>", continue_selection)
output_text.bind("<ButtonRelease-1>", end_selection)
output_text.tag_configure("highlight", background="yellow")
output_text.pack(side="top", fill="both", expand=True)
#add a title to the text area: 
label = tk.Label(root,text = "lyrics and chords editing area")
label.pack(side="top")
#add parsing button on bottom of main text window: 
paste_and_parse_button = tk.Button(output_frame, text="Paste and Parse", command=on_paste_and_parse)
paste_and_parse_button.pack(side="bottom",fill="both", expand=False)
#add a saving button below it: 
save_button = tk.Button(output_frame, text="Save", command=save_text)
save_button.pack(side="bottom", fill="both", expand=False)

# create the helper window (it display an easy to choose chords, the unique chord lines in the edited text, and allow for marking and transposing chords on lines)'
root2 = tk.Tk()
root2.title('Common Chords List')

# set the window dimensions and position
screen_width = root2.winfo_screenwidth()
screen_height = root2.winfo_screenheight()
width = 650
height = 400  # increased height to make space for the scrollable list
x = int(screen_width/2)+10
y = int((screen_height - height)/2)
root2.geometry(f'{width}x{height}+{x}+{y}')

colors = ['#e6194B', '#3cb44b', '#ffe119', '#0082c8']

# create a 4x4 table of labels for the number pad
for i in range(len(allchords)):
    for j in range(len(allchords[0])):
        color = colors[j//3]
        label = tk.Label(root2, bg=color, text=allchords[i][j], relief="solid", height=1, width=10)
        label.grid(row=i, column=j, padx=3, pady=3)
        label.bind("<Button-1>", lambda event: on_double_click(event, "chord_table"))

# create a new frame for the scrollable list
list_frame = tk.Frame(root2, bg="white", width=width, height=height//3)
list_frame.grid(row=len(allchords), column=0, columnspan=len(allchords[0]), sticky="nsew", padx=5, pady=5)

# create a scrolled text widget inside the list frame
used_chord_lines = tk.Listbox(list_frame, width=60, height=10, bg='light gray')
used_chord_lines.pack(side="left", fill="y")

# insert some sample text into the list box
used_chord_lines.insert("end", "used chord lines will be added to list this:")
used_chord_lines.bind("<Double-Button-1>", lambda event: on_double_click(event, "used_chord_lines"))

# configure the grid to have the same size for each element
for i in range(len(allchords)):
    root2.rowconfigure(i, weight=1)
    for j in range(len(allchords[0])):
        root2.columnconfigure(j, weight=1)

# make the last row (list frame) resizable
root2.rowconfigure(len(allchords), weight=1)

# create two small buttons near the listbox
lines_transpose_up_button = tk.Button(list_frame, text="Transpose [UP]",width=20)
lines_transpose_up_button.pack(side="top", pady=1,anchor='nw')

lines_transpose_down_button = tk.Button(list_frame, text="Transpose [DOWN]",width=20)
lines_transpose_down_button.pack(side="top", pady=1,anchor='nw')
# create two small buttons near the listbox

lines_transpose_up_button.bind("<Button-1>", lambda event: on_transpose_click(event, "lines_transpose_up_button",'up'))
lines_transpose_down_button.bind("<Button-1>", lambda event: on_transpose_click(event, "lines_transpose_down_button",'down'))

def close_windows():
    root.quit
    root2.quit
    root.destroy()
    root2.destroy()

root.protocol("WM_DELETE_WINDOW", close_windows)
root2.protocol("WM_DELETE_WINDOW", close_windows)
root2.focus_force()
root.focus_force()

root.mainloop()
root2.mainloop()


detected hebrew
selection start-end:  ('2.11', '2.11')
selection from chord_table
selected chord: Bb
current state of chords list added by user []
last chord length in list: 2
selection location on the grid ('2', '11')
location_on_string: (1, 10)
selected text line string _________________________
chord left of cursor:  
reordering suggested chords list
predicted chords ['F', 'C', 'Dm', 'Bb', 'C7', 'Eb', 'A', 'Am', 'G', 'BbM7', 'Gm', 'Gm7', 'A7', 'Cm', 'F7']
selection start-end:  ('2.4', '2.11')
selection from chord_table
selected chord: Bm
current state of chords list added by user ['Bb']
last chord length in list: 2
selection location on the grid ('2', '5')
location_on_string: (1, 4)
selected text line string _________Bb______________
chord left of cursor:  
reordering suggested chords list
predicted chords ['Bb', 'A', 'C', 'Dm', 'F', 'D', 'G', 'Gm', 'Eb', 'Am', 'A7', 'Bm', 'Dm7', 'Cm', 'Db']
selection start-end:  ('4.11', '2.11')
selection from chord_table
selected chord: Cm
current

In [8]:
get_chord_list=False
if get_chord_list: 

    import pandas as pd
    all_chords_positions=pd.read_csv('guitarchords.csv')

    all_chords_positions['lp_name_standard']=all_chords_positions['lp_name'].apply(lambda x:x.replace(':','').replace(',','').replace('_','').replace("'",'').replace('.',''))
    all_chords_positions['lp_name_standard']=all_chords_positions['lp_name_standard'].apply(lambda x:x.replace('maj','Maj'))

    def captialize_first(string_to_cap):
        cap_first=string_to_cap
        cap_first=cap_first[0].upper()+cap_first[1:]
        return cap_first

    all_chords_positions['lp_name_standard_cap']=all_chords_positions['lp_name_standard'].apply(captialize_first)
    print('table chords',all_chords_positions['lp_name_standard_cap'].unique())
    print('program chords',np.sort(list(encoder.number_to_category_dict.values())))
    